# About this notebook  
- PyTorch RNN starter code with W&B  
- Pytorch W&B Usage Examples from https://docs.wandb.ai/guides/integrations/pytorch  

If this notebook is helpful, feel free to upvote :)

![](https://raw.githubusercontent.com/google/deluca-lung/main/assets/2020-10-02%20Ventilator%20diagram.svg)

In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [21]:
from defs import *

In [22]:
# ====================================================
# CFG
# ====================================================
class CFG:
    competition='ventilator'
    _wandb_kernel='nakama'
    apex=False
    print_freq=100
    num_workers=10
    model_name='rnn'
    scheduler='CosineAnnealingLR' # ['linear', 'cosine', 'ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    batch_scheduler=False
    #num_warmup_steps=100 # ['linear', 'cosine']
    #num_cycles=0.5 # 'cosine'
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=50 # CosineAnnealingLR
    #T_0=50 # CosineAnnealingWarmRestarts
    epochs=6
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=64
    lr=7e-3
    min_lr=2e-6
    weight_decay=1e-6
    batch_size=128
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    cate_seq_cols=['R', 'C']
    cont_seq_cols=['time_step', 'u_in', 'u_out'] + ['breath_time', 'u_in_time']
    train=True
    inference=True

In [4]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import json
import time
import math
import random
from datetime import datetime
from collections import Counter, defaultdict

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from tqdm.auto import tqdm
import category_encoders as ce

from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")

if CFG.apex:
    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
# ====================================================
# wandb
# ====================================================
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

    
def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

run = wandb.init(project="Ventilator-Pressure-Public", 
                 name=CFG.model_name,
                 config=class2dict(CFG),
                 group=CFG.model_name,
                 job_type="train",
                 anonymous=anony)

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


wandb: Wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [24]:
# ====================================================
# Utils
# ====================================================
def get_score(y_trues, y_preds):
    score = mean_absolute_error(y_trues, y_preds)
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything()

In [25]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sample_submission.csv')

for c in ['u_in']:
    train[c] = np.log1p(train[c])
    test[c] = np.log1p(test[c])
    
r_map = {5: 0, 20: 1, 50: 2}
c_map = {10: 0, 20: 1, 50: 2}
train['R'] = train['R'].map(r_map)
test['R'] = test['R'].map(r_map)
train['C'] = train['C'].map(c_map)
test['C'] = test['C'].map(c_map)

display(train.head())
display(test.head())
display(sub.head())

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,1,2,0.000000,0.080043,0,5.837492
1,2,1,1,2,0.033652,2.964399,0,5.907794
2,3,1,1,2,0.067514,3.157395,0,7.876254
3,4,1,1,2,0.101542,3.170056,0,11.742872
4,5,1,1,2,0.135756,3.271690,0,12.234987


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,0,1,0.000000,0.000000,0
1,2,0,0,1,0.031904,2.141835,0
2,3,0,0,1,0.063827,2.750578,0
3,4,0,0,1,0.095751,3.101470,0
4,5,0,0,1,0.127644,3.307654,0


,id,pressure
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [26]:
# ====================================================
# FE
# ====================================================
def add_feature(df):
    # breath_time
    df['breath_time'] = df['time_step'] - df['time_step'].shift(1)
    df.loc[df['time_step'] == 0, 'breath_time'] = 0
    # u_in_time
    df['u_in_time'] = df['u_in'] - df['u_in'].shift(1)
    df.loc[df['time_step'] == 0, 'u_in_time'] = 0
    return df


train = add_feature(train)
test = add_feature(test)

In [27]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=5)
groups = train['breath_id'].values
for n, (train_index, val_index) in enumerate(Fold.split(train, train['pressure'], groups)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
print(train.groupby('fold').size())

fold
0    1207200
1    1207200
2    1207200
3    1207200
4    1207200
dtype: int64


In [28]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.hidden_size = cfg.hidden_size
        self.r_emb = nn.Embedding(3, 2, padding_idx=0)
        self.c_emb = nn.Embedding(3, 2, padding_idx=0)
        self.seq_emb = nn.Sequential(
            nn.Linear(4 + len(cfg.cont_seq_cols), self.hidden_size),
            nn.LayerNorm(self.hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, 
                            dropout=0.2, batch_first=True, bidirectional=True)
        self.head = nn.Sequential(
            nn.Linear(self.hidden_size * 2, self.hidden_size * 2),
            nn.LayerNorm(self.hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(0.),
            nn.Linear(self.hidden_size * 2, 1),
        )
        for n, m in self.named_modules():
            if isinstance(m, nn.LSTM):
                print(f'init {m}')
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)
            elif isinstance(m, nn.GRU):
                print(f"init {m}")
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        init.orthogonal_(param.data)
                    else:
                        init.normal_(param.data)

    def forward(self, cate_seq_x, cont_seq_x):
        bs = cont_seq_x.size(0)
        r_emb = self.r_emb(cate_seq_x[:,:,0]).view(bs, 80, -1)
        c_emb = self.c_emb(cate_seq_x[:,:,1]).view(bs, 80, -1)
        seq_x = torch.cat((r_emb, c_emb, cont_seq_x), 2)
        seq_emb = self.seq_emb(seq_x)
        seq_emb, _ = self.lstm(seq_emb)
        output = self.head(seq_emb).view(bs, -1)
        return output

In [29]:
# ====================================================
# helper function
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    losses = AverageMeter()
    start = end = time.time()
    for step, (cate_seq_x, cont_seq_x, u_out, y) in enumerate(train_loader):
        loss_mask = u_out == 0
        cate_seq_x, cont_seq_x, y = cate_seq_x.to(device), cont_seq_x.to(device), y.to(device)
        batch_size = cont_seq_x.size(0)
        pred = model(cate_seq_x, cont_seq_x)
        loss = 2. * criterion(pred[loss_mask], y[loss_mask]) + criterion(pred[loss_mask == 0], y[loss_mask == 0])
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader),
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   loss=losses,
                   grad_norm=grad_norm,
                   lr=scheduler.get_lr()[0],
                   ))
        wandb.log({f"[fold{fold}] loss": losses.val,
                   f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    preds = []
    losses = AverageMeter()
    start = end = time.time()
    for step, (cate_seq_x, cont_seq_x, u_out, y) in enumerate(valid_loader):
        loss_mask = u_out == 0
        cate_seq_x, cont_seq_x, y = cate_seq_x.to(device), cont_seq_x.to(device), y.to(device)
        batch_size = cont_seq_x.size(0)
        with torch.no_grad():
            pred = model(cate_seq_x, cont_seq_x)
        loss = 2. * criterion(pred[loss_mask], y[loss_mask]) + criterion(pred[loss_mask == 0], y[loss_mask == 0])
        losses.update(loss.item(), batch_size)
        preds.append(pred.view(-1).detach().cpu().numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader),
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   loss=losses,
                   ))
    preds = np.concatenate(preds)
    return losses.avg, preds


def inference_fn(test_loader, model, device):
    model.eval()
    model.to(device)
    preds = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (cate_seq_x, cont_seq_x) in tk0:
        cate_seq_x, cont_seq_x = cate_seq_x.to(device), cont_seq_x.to(device)
        with torch.no_grad():
            pred = model(cate_seq_x, cont_seq_x)
        preds.append(pred.view(-1).detach().cpu().numpy())
    preds = np.concatenate(preds)
    return preds

In [30]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    y_true = valid_folds['pressure'].values
    non_expiratory_phase_val_idx = valid_folds[valid_folds['u_out'] == 0].index # The expiratory phase is not scored

    train_dataset = TrainDataset(train_folds)
    valid_dataset = TrainDataset(valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG)
    model.to(device)
    model = nn.DataParallel(model)

    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    
    def get_scheduler(optimizer):
        if CFG.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif CFG.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=CFG.num_cycles
            )
        elif CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    scheduler = get_scheduler(optimizer)

    # ====================================================
    # apex
    # ====================================================
    if CFG.apex:
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.L1Loss()

    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(y_true[non_expiratory_phase_val_idx], preds[non_expiratory_phase_val_idx])

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - MAE Score (without expiratory phase): {score:.4f}')
        wandb.log({f"[fold{fold}] epoch": epoch+1, 
                   f"[fold{fold}] avg_train_loss": avg_loss, 
                   f"[fold{fold}] avg_val_loss": avg_val_loss,
                   f"[fold{fold}] score": score})
        
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'preds': preds},
                        OUTPUT_DIR+f"fold{fold}_best.pth")
            
    preds = torch.load(OUTPUT_DIR+f"fold{fold}_best.pth", map_location=torch.device('cpu'))['preds']
    valid_folds['preds'] = preds

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [31]:
# ====================================================
# main
# ====================================================
def main():
    
    """
    Prepare: 1.train 2.test
    """
    
    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df['pressure'].values
        non_expiratory_phase_val_idx = result_df[result_df['u_out'] == 0].index # The expiratory phase is not scored
        score = get_score(labels[non_expiratory_phase_val_idx], preds[non_expiratory_phase_val_idx])
        LOGGER.info(f'Score (without expiratory phase): {score:<.4f}')
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)
    
    if CFG.inference:
        test_dataset = TestDataset(test)
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size * 2, shuffle=False, num_workers=CFG.num_workers, pin_memory=True)
        for fold in CFG.trn_fold:
            model = CustomModel(CFG)
            path = OUTPUT_DIR+f"fold{fold}_best.pth"
            state = torch.load(path, map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
            predictions = inference_fn(test_loader, model, device)
            test[f'fold{fold}'] = predictions
            del state, predictions; gc.collect()
            torch.cuda.empty_cache()
        # submission
        test['pressure'] = test[[f'fold{fold}' for fold in range(CFG.n_fold)]].mean(1)
        test[['id', 'pressure']+[f'fold{fold}' for fold in range(CFG.n_fold)]].to_csv(OUTPUT_DIR+'raw_submission.csv', index=False)
        test[['id', 'pressure']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
    
    wandb.finish()

In [ ]:
if __name__ == '__main__':
    main()

========== fold: 0 training ==========
========== fold: 0 training ==========


init LSTM(64, 64, batch_first=True, dropout=0.2, bidirectional=True)
Epoch: [1][0/471] Elapsed 2m 9s (remain 1018m 6s) Loss: 43.5201(43.5201) Grad: 30.7354  LR: 0.007000  


wandb: Wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch: [1][100/471] Elapsed 2m 17s (remain 8m 25s) Loss: 5.8380(16.0565) Grad: 10.6756  LR: 0.007000  
Epoch: [1][200/471] Elapsed 2m 24s (remain 3m 14s) Loss: 4.1891(10.5941) Grad: 10.7566  LR: 0.007000  
Epoch: [1][300/471] Elapsed 2m 31s (remain 1m 25s) Loss: 4.0067(8.3990) Grad: 42.6261  LR: 0.007000  
Epoch: [1][400/471] Elapsed 2m 38s (remain 0m 27s) Loss: 3.8916(7.2296) Grad: 40.8703  LR: 0.007000  
Epoch: [1][470/471] Elapsed 2m 42s (remain 0m 0s) Loss: 3.9379(6.6871) Grad: 58.5869  LR: 0.007000  
EVAL: [0/118] Elapsed 1m 18s (remain 152m 23s) Loss: 3.0252(3.0252) 
EVAL: [100/118] Elapsed 1m 23s (remain 0m 14s) Loss: 2.9052(2.9754) 
EVAL: [117/118] Elapsed 1m 24s (remain 0m 0s) Loss: 2.5755(2.9800) 


Epoch 1 - avg_train_loss: 6.6871  avg_val_loss: 2.9800  time: 251s
Epoch 1 - avg_train_loss: 6.6871  avg_val_loss: 2.9800  time: 251s
Epoch 1 - MAE Score (without expiratory phase): 1.2598
Epoch 1 - MAE Score (without expiratory phase): 1.2598
Epoch 1 - Save Best Score: 1.2598 Model
Epoch 1 - Save Best Score: 1.2598 Model


...

Epoch 1 - avg_train_loss: 6.7862  avg_val_loss: 3.3926  time: 171s
Epoch 1 - avg_train_loss: 6.7862  avg_val_loss: 3.3926  time: 171s
Epoch 1 - MAE Score (without expiratory phase): 1.4460
Epoch 1 - MAE Score (without expiratory phase): 1.4460
Epoch 1 - Save Best Score: 1.4460 Model
Epoch 1 - Save Best Score: 1.4460 Model


Epoch: [2][0/471] Elapsed 1m 16s (remain 601m 42s) Loss: 3.5906(3.5906) Grad: 27.7806  LR: 0.006986  
Epoch: [2][100/471] Elapsed 1m 24s (remain 5m 10s) Loss: 3.0873(3.2776) Grad: 15.4336  LR: 0.006986  
Epoch: [2][200/471] Elapsed 1m 31s (remain 2m 2s) Loss: 3.0815(3.3395) Grad: 20.0429  LR: 0.006986  
Epoch: [2][300/471] Elapsed 1m 38s (remain 0m 55s) Loss: 2.7859(3.2387) Grad: 12.2306  LR: 0.006986  
Epoch: [2][400/471] Elapsed 1m 45s (remain 0m 18s) Loss: 2.7601(3.1938) Grad: 14.7065  LR: 0.006986  
Epoch: [2][470/471] Elapsed 1m 50s (remain 0m 0s) Loss: 3.0620(3.1646) Grad: 17.5896  LR: 0.006986  
EVAL: [0/118] Elapsed 1m 13s (remain 143m 27s) Loss: 3.0321(3.0321) 
EVAL: [100/118] Elapsed 1m 18s (remain 0m 13s) Loss: 2.6708(2.7413) 
EVAL: [117/118] Elapsed 1m 19s (remain 0m 0s) Loss: 2.4293(2.7530) 


Epoch 2 - avg_train_loss: 3.1646  avg_val_loss: 2.7530  time: 194s
Epoch 2 - avg_train_loss: 3.1646  avg_val_loss: 2.7530  time: 194s
Epoch 2 - MAE Score (without expiratory phase): 1.1349
Epoch 2 - MAE Score (without expiratory phase): 1.1349
Epoch 2 - Save Best Score: 1.1349 Model
Epoch 2 - Save Best Score: 1.1349 Model


Epoch: [3][0/471] Elapsed 1m 40s (remain 788m 11s) Loss: 3.0527(3.0527) Grad: 11.0611  LR: 0.006952  
Epoch: [3][100/471] Elapsed 1m 49s (remain 6m 42s) Loss: 2.9265(2.8914) Grad: 14.1906  LR: 0.006952  
Epoch: [3][200/471] Elapsed 1m 57s (remain 2m 38s) Loss: 2.8296(2.8452) Grad: 35.0611  LR: 0.006952  
Epoch: [3][300/471] Elapsed 2m 6s (remain 1m 11s) Loss: 2.9877(2.8402) Grad: 30.7687  LR: 0.006952  
Epoch: [3][400/471] Elapsed 2m 17s (remain 0m 23s) Loss: 2.7890(2.8169) Grad: 24.9383  LR: 0.006952  
Epoch: [3][470/471] Elapsed 2m 23s (remain 0m 0s) Loss: 2.6669(2.8046) Grad: 28.9782  LR: 0.006952  
EVAL: [0/118] Elapsed 1m 21s (remain 159m 8s) Loss: 2.5177(2.5177) 
EVAL: [100/118] Elapsed 1m 27s (remain 0m 14s) Loss: 2.2802(2.4091) 
EVAL: [117/118] Elapsed 1m 27s (remain 0m 0s) Loss: 2.1466(2.4212) 


Epoch 3 - avg_train_loss: 2.8046  avg_val_loss: 2.4212  time: 237s
Epoch 3 - avg_train_loss: 2.8046  avg_val_loss: 2.4212  time: 237s
Epoch 3 - MAE Score (without expiratory phase): 1.0113
Epoch 3 - MAE Score (without expiratory phase): 1.0113
Epoch 3 - Save Best Score: 1.0113 Model
Epoch 3 - Save Best Score: 1.0113 Model
